In [ ]:
!pip install facenet-pytorch matplotlib

In [ ]:
# 원본 사진 파일 불러오기
from google.colab import files
uploaded = files.upload()
!unzip -q part1.zip -d ./part1

In [ ]:
import os
from PIL import Image
from facenet_pytorch import MTCNN
import torch

# MTCNN 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)

In [ ]:
# 이미지 폴더
img_folder = 'part1'
img_files = [f for f in os.listdir(img_folder) if f.lower().endswith('.jpg')]

# 출력 폴더들
label_folder = 'labels'
crop_folder = 'crops'
os.makedirs(label_folder, exist_ok=True)
os.makedirs(crop_folder, exist_ok=True)

# 실패한 파일 기록용 리스트
failed_files = []

In [ ]:
for img_name in img_files:
    img_path = os.path.join(img_folder, img_name)
    img = Image.open(img_path).convert('RGB')
    w, h = img.size

    try:
        boxes, probs = mtcnn.detect(img)  # 얼굴 위치와 confidence score 반환

        # === confidence 기준 적용 ===
        min_conf = 0.9  # 기준이 너무 엄격하면 아래에서 fallback 수행
        if boxes is not None and probs is not None:
            filtered_boxes = [box for box, prob in zip(boxes, probs) if prob is not None and prob > min_conf]
        else:
            filtered_boxes = None

        # fallback: 너무 엄격해서 아무것도 감지되지 않았을 경우 완화된 기준으로 재시도
        if not filtered_boxes:
            min_conf = 0.6  # 완화된 기준
            if boxes is not None and probs is not None:
                filtered_boxes = [box for box, prob in zip(boxes, probs) if prob is not None and prob > min_conf]

        boxes = filtered_boxes if filtered_boxes else None

        if boxes is not None:
            # 라벨 저장
            label_path = os.path.join(label_folder, img_name.replace('.jpg', '.txt'))
            with open(label_path, 'w') as f:
                for box in boxes:
                    x1, y1, x2, y2 = box
                    x_center = ((x1 + x2) / 2) / w
                    y_center = ((y1 + y2) / 2) / h
                    box_w = (x2 - x1) / w
                    box_h = (y2 - y1) / h
                    f.write(f"0 {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}\n")

            # 얼굴 crop 저장
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = map(int, box)
                face = img.crop((x1, y1, x2, y2))
                face_filename = f"{img_name.replace('.jpg', f'_face{i+1}.jpg')}"
                face.save(os.path.join(crop_folder, face_filename))
        else:
            print(f"얼굴 감지 실패: {img_name}")
            failed_files.append(img_name)

    except Exception as e:
        print(f"오류 발생: {img_name}, 오류 메시지: {str(e)}")
        failed_files.append(img_name)

In [ ]:
# 요약 출력
print("\n=== 처리 요약 ===")
print(f"총 {len(failed_files)}개 파일에서 얼굴 감지 실패 또는 오류 발생")
if failed_files:
    print("실패 파일 목록:")
    for name in failed_files:
        print(" -", name)

In [ ]:
# label 저장
!zip -r part1_labels.zip labels
from google.colab import files
files.download('part1_labels.zip')

# crop된 사진 저장
!zip -r part1_crops.zip crops
files.download('part1_crops.zip')